In [35]:
import time
import os

path = '/Users/viedamayanti/Desktop/Data_file/Combined_Flights_2021.csv'


In [36]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv(path)
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.07381987571716309 sec


In [37]:
import pandas as pd
start = time.time()
pandas_df = pd.read_csv(path)
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  86.5959050655365 sec


I have difficulties using Modin and Ray, so I stick to Dask and Pandas. However, between these two, Dask performs better than Pandas. Therefore, I am using Dask

In [38]:
df = dd.read_csv(path)
df.head(3)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2021-03-03,SkyWest Airlines Inc.,SGU,PHX,False,False,724,714.0,0.0,-10.0,...,724.0,813.0,5.0,843,-25.0,0.0,-2.0,0800-0859,2,0.0
1,2021-03-03,SkyWest Airlines Inc.,PHX,SGU,False,False,922,917.0,0.0,-5.0,...,940.0,1028.0,3.0,1040,-9.0,0.0,-1.0,1000-1059,2,0.0
2,2021-03-03,SkyWest Airlines Inc.,MHT,ORD,False,False,1330,1321.0,0.0,-9.0,...,1336.0,1445.0,16.0,1530,-29.0,0.0,-2.0,1500-1559,4,0.0


In [39]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 61 entries, FlightDate to DivAirportLandings
dtypes: object(18), bool(2), float64(19), int64(22)

In [40]:
df.columns

Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOu

In [41]:
df.columns.isnull().sum()

0

In [42]:
import re
def special_characters(value):
    return bool(re.search(r'[^a-zA-Z0-9\s]', str(value)))

special_characters_check = df.columns.map(special_characters)
print(special_characters_check.value_counts())

False    51
True     10
Name: count, dtype: int64


In [43]:
%%writefile flight_data.yaml
file_type: csv
file_name: Combined_Flights_2021
inbound_delimiter: ","
outbound_delimiter: "|"
columns:
- FlightDate
- Airline
- Origin
- Dest
- Cancelled
- Diverted
- CRSDepTime
- DepTime
- DepDelayMinutes
- DepDelay
- ArrTime
- ArrDelayMinutes
- AirTime
- CRSElapsedTime
- ActualElapsedTime
- Distance
- Year
- Quarter
- Month
- DayofMonth
- DayOfWeek
- Marketing_Airline_Network
- Operated_or_Branded_Code_Share_Partners
- DOT_ID_Marketing_Airline
- IATA_Code_Marketing_Airline
- Flight_Number_Marketing_Airline
- Operating_Airline
- DOT_ID_Operating_Airline
- IATA_Code_Operating_Airline
- Tail_Number
- Flight_Number_Operating_Airline
- OriginAirportID
- OriginAirportSeqID
- OriginCityMarketID
- OriginCityName
- OriginState
- OriginStateFips
- OriginStateName
- OriginWac
- DestAirportID
- DestAirportSeqID
- DestCityMarketID
- DestCityName
- DestState
- DestStateFips
- DestStateName
- DestWac
- DepDel15
- DepartureDelayGroups
- DepTimeBlk
- TaxiOut
- WheelsOff
- WheelsOn
- TaxiIn
- CRSArrTime
- ArrDelay
- ArrDel15
- ArrivalDelayGroups
- ArrTimeBlk
- DistanceGroup
- DivAirportLandings
file_path: /Users/viedamayanti/Desktop/Data_file/Combined_Flights_2021.csv


Overwriting flight_data.yaml


In [44]:
file_path = '/Users/viedamayanti/Desktop/Data-Glacier-Data-Analyst-Intern/Week6/flight_data.yaml'


In [45]:
import yaml
import logging
def read_file(file_path):
    with open(file_path,'r') as stream:
        try:
            yaml_data = yaml.safe_load(stream)
            return yaml_data
        except yaml.YAMLError as exc:
            print(f"Error reading YAML file: {exc}")
            logging.error(exc)
           

def col_validation(df_columns, yaml_data):
    expected_col = yaml_data.get('columns')
    if len(df.columns) == len(expected_col):
        print('Validation passed')
        return 1
    else:
        print('Column not matched - validation failed')
        logging.info(f'df columns: {df_columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

# Read the YAML file
loaded_data = read_file(file_path)
if loaded_data:
    result = col_validation(df.columns, loaded_data)
    print(result)
else:
    print('Error YAML file')


Validation passed
1


In [46]:
config_data = read_file(file_path)
print(config_data['file_name'])

Combined_Flights_2021


In [47]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted',
       'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime',
       'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime',
       'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline',
       'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline',
       'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15',
       'DepartureDelayGroups',

In [48]:
# File summary
file_size = os.path.getsize(path)
num_rows = len(df)
num_columns = len(df.columns)

# Print the file summary 
print('File Summary: ')
print(f'Size of the file: {file_size}')
print(f'Number of rows: {num_rows}')
print(f'Number of columns: {num_columns}')

File Summary: 
Size of the file: 2214293351
Number of rows: 6311871
Number of columns: 61


In [49]:
bytes_value = file_size
gigabytes_value = bytes_value / (1024**3)

print(f"{bytes_value} bytes is approximately {gigabytes_value:.6f} gigabytes.")

2214293351 bytes is approximately 2.062221 gigabytes.
